In [1]:
using Turing
using Distributions
using DataFrames
using Random

# 再現性のためにシードを設定
Random.seed!(123)

# 現実的なサンプルデータを生成
n_samples = 1000

# 特徴量1: チーム力 (正規分布)
team_strength = randn(n_samples) * 2 # 平均0, 標準偏差2

# 特徴量2: ホームアドバンテージ (二値: 0または1)
home_advantage = rand([0, 1], n_samples)

# 特徴量3: オッズ (べき乗則分布、チーム力とホームアドバンテージに基づいて調整)
odds = zeros(n_samples)
for i in 1:n_samples
    base_odds = 2.0 + abs(team_strength[i]) * 0.5 # 基本オッズはチーム力の影響を受ける
    if home_advantage[i] == 1
        base_odds *= 0.8 # ホームアドバンテージはオッズを減らす (勝利の可能性を高める)
    end
    odds[i] = rand(Pareto(1.5, base_odds)) # スケールが基本オッズの影響を受けるパレート分布
    # 極端に高い値を避けるためにオッズに上限を設定
    odds[i] = min(odds[i], 20.0)
end

# 目的変数: 勝利 (二値: 0または1)
win = zeros(Int, n_samples)
for i in 1:n_samples
    win_prob = 1 / odds[i] # オッズに基づく勝利確率
    win[i] = rand() < win_prob ? 1 : 0 # 勝敗をシミュレート
end

df = DataFrame(TeamStrength = team_strength, HomeAdvantage = home_advantage, Odds = odds, Win = win)

# Turingモデル
@model function betting_model(team_strength, home_advantage, odds, win)
    # 事前分布
    β_strength ~ Normal(0, 1) # チーム力の影響
    β_home ~ Normal(0, 1) # ホームアドバンテージの影響
    α ~ truncated(Pareto(1, 1), lower = 1.0) #パレート分布の形状パラメータ
    # σ ~ Exponential(1) # 線形部分のノイズ (オッズに直接使用されない)

    # 尤度
    for i in 1:length(win)
        # 基本オッズのモデル (パレート分布の前)
        base_odds = exp(β_strength * team_strength[i] + β_home * home_advantage[i])
        odds[i] ~ Pareto(α, base_odds)

        # 勝利確率のモデル (シミュレートされたオッズを使用)
        win_prob = 1 / odds[i]
        win[i] ~ Bernoulli(win_prob)
    end
    return β_strength, β_home, α
end

# モデルを作成してサンプリング
model = betting_model(df.TeamStrength, df.HomeAdvantage, df.Odds, df.Win)
chain = sample(model, NUTS(0.65), 1000)

display(chain)

# 新しいデータの予測
@model function prediction_model(team_strength_new, home_advantage_new, α, β_strength, β_home)
    base_odds = exp(β_strength * team_strength_new + β_home * home_advantage_new)
    odds_new ~ Pareto(α, base_odds)
    win_prob = 1/odds_new
    win_new ~ Bernoulli(win_prob)
    return win_new
end

new_team_strength = 1.5
new_home_advantage = 1
prediction_chain = sample(prediction_model(new_team_strength,new_home_advantage, chain[:α], chain[:β_strength], chain[:β_home]),1000)
println("Probability of win: ", mean(prediction_chain[:win_new]))

┌ Info: Found initial step size
└   ϵ = 0.0125
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:04


Chains MCMC chain (1000×15×1 Array{Float64, 3}):

Iterations        = 501:1:1500
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 12.31 seconds
Compute duration  = 12.31 seconds
parameters        = β_strength, β_home, α
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std      mcse   ess_bulk   ess_tail      rhat   e ⋯
      Symbol   Float64   Float64   Float64    Float64    Float64   Float64     ⋯

  β_strength   -0.1496    0.0074    0.0032     6.9059    12.8798    1.1789     ⋯
      β_home    0.4908    0.0017    0.0005    11.0840    14.5143    1.1219     ⋯
           α    1.1856    0.2035    0.1015     2.4187    12.2549    2.0638     ⋯
                                                                1 column omitted

Quantiles
  parameters      2.5%     25.0%     50.

LoadError: MethodError: no method matching sample(::DynamicPPL.Model{typeof(prediction_model), (:team_strength_new, :home_advantage_new, :α, :β_strength, :β_home), (), (), Tuple{Float64, Int64, AxisArrays.AxisMatrix{Float64, Matrix{Float64}, Tuple{AxisArrays.Axis{:iter, StepRange{Int64, Int64}}, AxisArrays.Axis{:chain, UnitRange{Int64}}}}, AxisArrays.AxisMatrix{Float64, Matrix{Float64}, Tuple{AxisArrays.Axis{:iter, StepRange{Int64, Int64}}, AxisArrays.Axis{:chain, UnitRange{Int64}}}}, AxisArrays.AxisMatrix{Float64, Matrix{Float64}, Tuple{AxisArrays.Axis{:iter, StepRange{Int64, Int64}}, AxisArrays.Axis{:chain, UnitRange{Int64}}}}}, Tuple{}, DynamicPPL.DefaultContext}, ::Int64)
The function `sample` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  sample([91m::AbstractRNG[39m, ::Any, [91m::AbstractMCMC.AbstractSampler[39m, [91m::Any[39m; kwargs...)
[0m[90m   @[39m [33mAbstractMCMC[39m [90m~/.julia/packages/AbstractMCM